<a href="https://www.kaggle.com/code/abukhadeejahkarl/imdb-star-wars-analysis?scriptVersionId=143565366" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os



# IMDB Non-Commercial Datasets
Subsets of IMDB data are available for access to customers for personal and non-commercial use. You can hold local copies of this data, and it is subject to our terms and conditions. Please refer to the [Non-Commercial Licensing](https://help.imdb.com/article/imdb/general-information/can-i-use-imdb-data-in-my-software/G5JTRESSHJBBHTGX?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=1ed1aea6-d2ad-4705-95fd-ba13f1b5014f&pf_rd_r=XRE3QWF2G5YWTD2SGT0V&pf_rd_s=center-1&pf_rd_t=60601&pf_rd_i=interfaces&ref_=fea_mn_lk1) and [copyright/license](http://www.imdb.com/Copyright?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=1ed1aea6-d2ad-4705-95fd-ba13f1b5014f&pf_rd_r=XRE3QWF2G5YWTD2SGT0V&pf_rd_s=center-1&pf_rd_t=60601&pf_rd_i=interfaces&ref_=fea_mn_lk2) and verify compliance.

# Data Location
The dataset files can be accessed and downloaded from https://datasets.imdbws.com/. The data is refreshed daily.

# IMDB Dataset Details
Each dataset is contained in a gzipped, tab-separated-values (TSV) formatted file in the UTF-8 character set. The first line in each file contains headers that describe what is in each column. A ‘\N’ is used to denote that a particular field is missing or null for that title/name.

In [ ]:
#main media name dataset
tbasics_file = "/kaggle/input/imdb-basic-dataset/title.basics.tsv/data.tsv"
# TV show link table
eposode_file = "/kaggle/input/imdb-basic-dataset/title.episode.tsv/data.tsv"
# Big file, a linking dataset for film workers/acters to videos.  also lists jobs, category
principals_file = "/kaggle/input/imdb-basic-dataset/title.principals.tsv/data.tsv"
# ratings
ratings_file = "/kaggle/input/imdb-basic-dataset/title.ratings.tsv/data.tsv"
# Actors, Workers, people in credits
nbasics_file = "/kaggle/input/imdb-basic-dataset/name.basics.tsv/data.tsv"
# very big file; alternative names for movies. 
#  some Japanese or Chinese moves have different titles in different regions
#  ie - Shogun Assassian (US) and Lone Wolf and Cub (JP)
akas_file = "/kaggle/input/imdb-basic-dataset/title.akas.tsv/data.tsv"
# link dataset for media to directors and writers
crew_file = "/kaggle/input/imdb-basic-dataset/title.crew.tsv/data.tsv"

# Loaded the Media names and ratings dataset
- this will also download my IMDB class for dealing with these datasets
- this is still a work in progress, but it will make using multiple notebooks easier to read and use.

In [ ]:
!rm -f MediaProc.py
!wget -O MediaProc.py https://raw.githubusercontent.com/salamcast/IMDBnotebook-MySQL/main/MediaProc.py

import MediaProc as mp
IM = mp.IMDB(tbasics_file, ratings_file)


# Make year values an integer values
**This will need to be moved into the MediaProc python class**


In [ ]:

IM.tbasics['startYear'] = IM.tbasics['startYear'].astype('int64', errors='ignore') #
IM.tbasics['endYear'] = IM.tbasics['endYear'].astype('int64', errors='ignore') #.convert_dtypes(convert_string=False)
IM.tbasics['runtimeMinutes'] = IM.tbasics['runtimeMinutes'].astype('int64', errors='ignore') #.convert_dtypes(convert_string=False)
IM.tbasics['numVotes'] = IM.tbasics['numVotes'].astype('int64', errors='ignore')
IM.tbasics

# Build TV show DataFrame
- Builds a new Videos DataFrame
- Builds a new TV Shows DataFrame

In [ ]:
IM.set_tvshows(eposode_file)


# Lets look at the ratings of Star Wars
I was 5-6 when I first saw a New Hope and Empire Strikes Back.  I also read a lot of the legacy books when I was in grade 8.  

lots of memories. 

I'm only selecting the main live action Movies, the Clone Wars anamated movie has less than 100000 votes, so that will be filtered out based on that.

In [ ]:
SW = IM.search_video_df("Star Wars", votes=100000)
SW

# lets look at the Average Rating and Number of Votes of the movies

My favorate Star Wars movie is Revenge of the Sith, it's IMDB rating is 7.6.  

My least favorate star wars movies are Last Jedi and The Rise of Skywalker, their IMDB ratings are 6.9 and 6.5.

Empire Strikes Back has the highest average rating, then A New Hope and Return of the Jedi -- these are Classics!


In [ ]:
mp.MovieAvgRating(SW)

In [ ]:
mp.MovieRuntime(SW)

# Scater Plot of Average Rating to number of votes over the years
**need to add this into MediaProc.py as a function**

In [ ]:
SW.plot.scatter(
    title="How did the average rating of Star Wars change over the years?\n\n",
    x='primaryTitle',
    c='numVotes',
    y='averageRating',
    colormap='viridis',
    rot=90,
    #logy=True, 
    figsize=(10, 10)
)

In [ ]:
mp.MovieHexBin(SW,title="Star Wars Movie Average Rating Vote comparison")

# What is the trend of Star Wars TV Shows?
im going to plot all the TV shows that have Star Wars in the TV show title.  this will look at the ratings by year and color the values based on votes for each entry. The newer series has a lot more votes than some of the older ones and they vary. The clone wars cartoon had a fews gap between the end of season 6 and 7, so it had lost some of the early viewer ship. Disney+ has made it easier to rewatch the old Star Wars movies and TV shows, but they have made some desions that have angered many subscribers enough to cancel their subscriptions.  

In [ ]:
#SWTV = IM.TVShow.loc[ (IM.TVShow.numVotes > 100 ) & IM.TVShow["TVShow"].str.contains("Star Wars|The Book of Boba Fett|Mandalorian|Ahsoka|Andor|Obi-Wan Kenobi", regex=True) ].drop(columns=['titleType', 'originalTitle', 'isAdult', "year","originalEpisode", "genres", "endYear"]).sort_values(by='startYear').dropna()
#SWTV['startYear'] = SWTV['startYear'].astype('int64', errors='ignore')
#SWTV['minutes'] = SWTV['minutes'].astype('int64', errors='ignore')
#SWTV['episodeTitle'] = SWTV['S'].astype('str') + 'x' + SWTV['E'].astype('str') + ' ' + SWTV['episodeTitle']
#SWTV['S'] = SWTV['S'].astype('int64', errors='ignore')
#SWTV['E'] = SWTV['E'].astype('int64', errors='ignore')
SWTV = IM.search_tvshow_df("Star Wars|The Book of Boba Fett|Mandalorian|Ahsoka|Andor|Obi-Wan Kenobi", votes=100)
SWTV

# Ploting the TV Series data

- Average Rating line plot
- Number of Votes line plot
- Run Time line plot
- HexBin

In [ ]:

mp.TVShowHexBin(SWTV,x='startYear', y='averageRating',c='numVotes', title="Star Wars TV Shows Average Rating over the years",
                 fig=(10, 10)
               )

# Lets look at the Star Wars TV Show data grouped by each Show
Each show has an average rating of all episode in each TV Show.
Sum of all the votes for each TV Show

In [ ]:
ep = SWTV.groupby('TVShow').E.count()

mins = SWTV.groupby('TVShow').minutes.sum()
rate = SWTV.groupby('TVShow').averageRating.sum()
votes = SWTV.groupby('TVShow').numVotes.sum()

enum = pd.DataFrame(ep)
enum['minutes'] = mins
enum['averageRating'] = rate / enum['E']
enum['numVotes'] = votes
enum = enum.sort_values(by="averageRating").reset_index()

In [ ]:
enum

In [ ]:
mp.TVShowAvgRating(enum, (10,10), "TVShow")

In [ ]:
mp.TVShowRuntime(enum, (10,10), "TVShow")

In [ ]:
mp.TVShowHexBin(enum, (10,10))

# Lets look at each the more recient Star Wars TV shows
I will look ate the more recient star wars tv shows and plot the data so you can see how they compaire over time based on Average Rating, Number of Votes and Run Time over their run.  The shows we will be looking at are:

- Star Wars: The Clone Wars
- Andor
- Star Wars: The Bad Batch
- Ahsoka
- The Mandalorian
- Obi-Wan Kenobi

The Clone Wars has the most episodes and ran over several years, it was also an animated series.  Many of the Star Wars TV Series before The Mandalorian have been Animated and aimed at a younger audience, like Rebels and The Bad Batch.

Something I have noticed with the new TV Series on Disney+ is that the run time is inconsistant with shows that were on TV in the past -- Star Trek DS9 would have episodes that ran around 45 minutes each

In [ ]:
shows = {}
shows['theCloneWars'] = SWTV.loc[(SWTV.TVShow == 'Star Wars: The Clone Wars')].sort_values(by=['S', 'E'])
shows['Andor'] = SWTV.loc[(SWTV.TVShow == 'Andor')].sort_values(by=['S', 'E'])
shows['BadBatch'] = SWTV.loc[(SWTV.TVShow == 'Star Wars: The Bad Batch')].sort_values(by=['S', 'E'])
shows['Ahsoka'] = SWTV.loc[(SWTV.TVShow == 'Ahsoka')].sort_values(by=['S', 'E'])
shows['Mando'] = SWTV.loc[(SWTV.TVShow == 'The Mandalorian')].sort_values(by=['S', 'E'])
shows['Obiwan'] = SWTV.loc[(SWTV.TVShow == 'Obi-Wan Kenobi')].sort_values(by=['S', 'E'])

# here is a quick view of The Clone Wars DataFrame
All the other DataFrames will have the same columns, so I will just plot them bellow.

In [ ]:
shows['theCloneWars']

# The Clone Wars
This a good anamated series, the average ratings jump around someware between 6 to 10, the 

In [ ]:
mp.TVShowAvgRating(shows['theCloneWars'], (25,10))

In [ ]:
mp.TVShowRuntime(shows['theCloneWars'], (25,10))

In [ ]:
mp.TVShowHexBin(shows['theCloneWars'], (10,10), 'the Clone Wars, Average Rating vs Number of votes')

In [ ]:
mp.TVShowAvgRating(shows['Andor'])


In [ ]:
mp.TVShowRuntime(shows['Andor'])

In [ ]:
mp.TVShowHexBin(shows['Andor'], title="Andor, Average Rating vs Number of votes")

In [ ]:
mp.TVShowAvgRating(shows['BadBatch'])

In [ ]:
mp.TVShowRuntime(shows['BadBatch'])

In [ ]:
mp.TVShowHexBin(shows['BadBatch'], title='The Bad Batch, Average Rating vs Number of votes')

In [ ]:
mp.TVShowAvgRating(shows['Ahsoka'])

In [ ]:
mp.TVShowRuntime(shows['Ahsoka'])

In [ ]:
mp.TVShowHexBin(shows['Ahsoka'], title='Ahsoka, Average Rating vs Number of votes')

In [ ]:
mp.TVShowAvgRating(shows['Mando'])


In [ ]:
mp.TVShowRuntime(shows['Mando'])

In [ ]:
mp.TVShowHexBin(shows['Mando'], title='The Mandalorian, Average Rating vs Number of votes')

In [ ]:
mp.TVShowAvgRating(shows['Obiwan'])


In [ ]:
mp.TVShowRuntime(shows['Obiwan'])

In [ ]:
mp.TVShowHexBin(shows['Obiwan'], title='Obiwan, Average Rating vs Number of votes')